# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

## 필요한 library 호출

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.python.ops import array_ops

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

tf.logging.set_verbosity(tf.logging.INFO)


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_txt = open('./data_split/5fold_0802.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 224
r_h = 224

total_pix = r_w * r_h * channel

In [4]:
tr_data_dir['train0'][0]

'/dataset_full/Torn/Torn (1220).png'

In [5]:
def vgg16(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, height, width, channels]
    # Our Fishing net image size is 640x480 and 3-channel (RGB)
    input_layer = tf.reshape(features["x"], [-1, 224, 224, 3])
    dropout_keep_prob = 0.5
    num_classes = 2
    is_training = True
    
    
    net = layers_lib.repeat(
        input_layer, 2, layers.conv2d, 64, [3, 3], scope='conv1')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool1')
    net = layers_lib.repeat(net, 2, layers.conv2d, 128, [3, 3], scope='conv2')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool2')
    net = layers_lib.repeat(net, 3, layers.conv2d, 256, [3, 3], scope='conv3')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool3')
    net = layers_lib.repeat(net, 3, layers.conv2d, 512, [3, 3], scope='conv4')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool4')
    net = layers_lib.repeat(net, 3, layers.conv2d, 512, [3, 3], scope='conv5')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool5')
    # Use conv2d instead of fully_connected layers.
    net = layers.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
    net = layers_lib.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout6')
    net = layers.conv2d(net, 4096, [1, 1], scope='fc7')
    net = layers_lib.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout7')
    net = layers.conv2d(
        net,
        num_classes, [1, 1],
        activation_fn=None,
        normalizer_fn=None,
        scope='fc8')
    net = array_ops.squeeze(net, [1, 2], name='fc8/squeezed')
    
    logits = net
    

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    # sparse_softmax_cross_entropy cannot use one-hot encoding
    
    #loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9
#session = tf.Session(config=config)

k = 5
total_res = {}

for i in range(k):
    val_result = []

    f_dir = "/models/ICROS_vgg/kfold_%d/" % i

    if not tf.gfile.Exists(f_dir):
        tf.gfile.MakeDirs(f_dir)

    net_classifier = tf.estimator.Estimator(
        model_fn=vgg16, model_dir=f_dir)

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    
    
    tr_data = np.zeros((len(tr_data_dir['train%d'%i]), total_pix), dtype=np.float32)
    tr_label = np.zeros((len(tr_data_dir['train%d'%i]), 1), dtype=np.int32)

    for j in range(len(tr_data)):
        img = cv2.imread(tr_data_dir['train%d'%i][j])
        img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
        tr_data[j,:] = img2.flatten()

        if 'Untorn' in tr_data_dir['train%d'%i][j]:
            tr_label[j] = 0
        elif 'normal' in tr_data_dir['train%d'%i][j]:
            tr_label[j] = 0

        else:
            tr_label[j] = 1

    
    val_data = np.zeros((len(tr_data_dir['valid%d'%i]), total_pix), dtype=np.float32)
    val_label = np.zeros((len(tr_data_dir['valid%d'%i]), 1), dtype=np.int32)

    for j in range(len(val_data)):
        img = cv2.imread(tr_data_dir['valid%d'%i][j])
        img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
        val_data[j,:] = img2.flatten()

        if 'Untorn' in tr_data_dir['valid%d'%i][j]:
            val_label[j] = 0
        elif 'normal' in tr_data_dir['valid%d'%i][j]:
            val_label[j] = 0

        else:
            val_label[j] = 1
            
            
            
            
    batch_s = 32
    epochs = 10
    tr_steps = int(tr_data.shape[0]/batch_s*epochs)
    in_steps = 1


    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": tr_data},
        y=tr_label,
        batch_size=batch_s,
        num_epochs=epochs,
        shuffle=False)
    
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": val_data},
        y=val_label,
        batch_size=64,
        num_epochs=1,
        shuffle=False)
    
    net_classifier.train(
        input_fn=train_input_fn,
        steps=tr_steps,
        hooks=[logging_hook])

    eval_results = net_classifier.evaluate(input_fn=eval_input_fn, hooks=[logging_hook])
    temp_res = eval_results["accuracy"]
    
    total_res['%d'%i] = temp_res

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_summary_steps': 100, '_is_chief': True, '_tf_random_seed': None, '_global_id_in_cluster': 0, '_master': '', '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_session_config': None, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_task_type': 'worker', '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa860888a58>, '_model_dir': '/models/ICROS_vgg/kfold_0/', '_evaluation_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /models/ICROS_vgg/kfold_0/model.ckpt.
INFO:tensorflow:probabilities = [[0.

INFO:tensorflow:loss = 0.3923307, step = 401 (33.517 sec)
INFO:tensorflow:probabilities = [[0.00000297 0.999997  ]
 [0.13782679 0.8621732 ]
 [0.20632786 0.79367214]
 [0.00429404 0.995706  ]
 [0.9999993  0.00000073]
 [1.         0.        ]
 [0.14947481 0.85052526]
 [0.30788767 0.6921123 ]
 [0.59990466 0.40009534]
 [0.00000204 0.999998  ]
 [0.99999595 0.00000401]
 [0.00407714 0.9959228 ]
 [0.16329893 0.83670104]
 [0.12203544 0.87796456]
 [0.14150871 0.85849124]
 [0.9999795  0.00002056]
 [0.00000519 0.99999475]
 [0.9263731  0.07362683]
 [0.06283534 0.93716466]
 [0.999433   0.00056694]
 [0.5641439  0.43585607]
 [1.         0.        ]
 [0.884774   0.115226  ]
 [0.9971328  0.00286723]
 [0.17774856 0.8222515 ]
 [1.         0.        ]
 [0.21854255 0.7814574 ]
 [0.8955796  0.10442041]
 [0.725395   0.27460498]
 [0.99351937 0.00648059]
 [0.86896944 0.1310305 ]
 [1.         0.        ]] (16.751 sec)
INFO:tensorflow:global_step/sec: 2.99475
INFO:tensorflow:probabilities = [[0.45430946 0.54569054

INFO:tensorflow:loss = 0.11379245, step = 901 (33.462 sec)
INFO:tensorflow:probabilities = [[0.58260083 0.4173992 ]
 [0.99973315 0.00026683]
 [0.03806124 0.96193874]
 [0.4009603  0.5990397 ]
 [0.00229987 0.99770015]
 [0.00034124 0.9996587 ]
 [0.9999845  0.00001553]
 [0.00143598 0.99856406]
 [1.         0.        ]
 [0.00693448 0.9930655 ]
 [0.12757619 0.8724238 ]
 [0.10393007 0.89606994]
 [0.00089839 0.99910164]
 [0.07505592 0.92494404]
 [0.00003516 0.99996483]
 [0.1446977  0.8553023 ]
 [0.14652571 0.8534743 ]
 [0.00036433 0.9996357 ]
 [0.9999958  0.00000421]
 [0.15460807 0.845392  ]
 [0.23748028 0.7625198 ]
 [0.02137602 0.978624  ]
 [0.00025672 0.9997433 ]
 [0.01278281 0.98721725]
 [0.7551513  0.24484868]
 [0.9999999  0.00000011]
 [0.98744106 0.012559  ]
 [0.00012202 0.9998779 ]
 [0.00736583 0.9926341 ]
 [0.05264251 0.9473575 ]
 [0.         1.        ]
 [0.99951684 0.00048314]] (16.755 sec)
INFO:tensorflow:global_step/sec: 2.9928
INFO:tensorflow:probabilities = [[1.         0.        

INFO:tensorflow:global_step/sec: 3.10894
INFO:tensorflow:probabilities = [[0.20915456 0.7908454 ]
 [0.640157   0.35984302]
 [0.7488317  0.25116828]
 [0.05002208 0.94997793]
 [0.62405443 0.37594554]
 [0.06550779 0.93449223]
 [0.6247717  0.3752283 ]
 [0.03087189 0.9691281 ]
 [0.6878155  0.3121845 ]
 [0.17886284 0.8211372 ]
 [0.6286898  0.3713102 ]
 [0.11577968 0.8842203 ]
 [0.6842864  0.3157136 ]
 [0.74992186 0.25007817]
 [0.02378134 0.9762187 ]
 [0.01427544 0.98572457]
 [0.6368784  0.36312157]
 [0.03292109 0.9670789 ]
 [0.08684029 0.91315967]
 [0.578675   0.42132506]
 [0.05272013 0.9472798 ]
 [0.751734   0.248266  ]
 [0.77200073 0.22799927]
 [0.6912881  0.30871186]
 [0.699184   0.30081603]
 [0.6801606  0.31983942]
 [0.22959128 0.77040875]
 [0.07642336 0.9235766 ]
 [0.65265954 0.3473405 ]
 [0.0406488  0.9593512 ]
 [0.6107183  0.38928166]
 [0.63012457 0.36987546]] (16.008 sec)
INFO:tensorflow:loss = 0.49993575, step = 101 (32.166 sec)
INFO:tensorflow:probabilities = [[0.90120226 0.0987977

INFO:tensorflow:global_step/sec: 3.07031
INFO:tensorflow:probabilities = [[0.00000191 0.9999981 ]
 [0.00471486 0.9952852 ]
 [0.00870667 0.9912934 ]
 [0.29438946 0.7056106 ]
 [0.00248514 0.99751484]
 [0.18456906 0.81543094]
 [0.00007713 0.9999229 ]
 [0.21918552 0.7808145 ]
 [0.00023372 0.9997663 ]
 [0.07038254 0.9296175 ]
 [0.99709535 0.00290467]
 [0.00000001 1.        ]
 [0.00000225 0.99999774]
 [0.923182   0.07681797]
 [0.00000036 0.99999964]
 [0.9999659  0.00003411]
 [0.00000597 0.99999404]
 [1.         0.00000001]
 [0.28509054 0.7149095 ]
 [0.36997324 0.63002676]
 [0.99990296 0.000097  ]
 [0.00033511 0.9996649 ]
 [0.         1.        ]
 [0.9852099  0.0147901 ]
 [1.         0.        ]
 [0.99999607 0.00000392]
 [0.42982826 0.5701718 ]
 [0.01400517 0.9859948 ]
 [0.15217355 0.8478264 ]
 [0.00000987 0.9999901 ]
 [0.9968194  0.00318056]
 [1.         0.00000003]] (16.328 sec)
INFO:tensorflow:loss = 0.09567936, step = 601 (32.571 sec)
INFO:tensorflow:probabilities = [[0.00000046 0.9999995

...

INFO:tensorflow:global_step/sec: 3.08187
INFO:tensorflow:probabilities = [[0.6043136  0.39568636]
 [0.54895455 0.45104545]
 [0.19189097 0.80810905]
 [0.7130118  0.28698817]
 [0.99999857 0.00000141]
 [0.33136275 0.6686373 ]
 [0.99999964 0.00000035]
 [0.20579264 0.7942074 ]
 [0.9999726  0.00002746]
 [1.         0.        ]
 [0.00009328 0.99990666]
 [0.00701418 0.9929858 ]
 [0.01106465 0.9889354 ]
 [0.19565302 0.804347  ]
 [0.18521239 0.81478757]
 [0.22165315 0.77834684]
 [0.14254603 0.857454  ]
 [0.23197454 0.7680255 ]
 [0.03528977 0.96471024]
 [0.00667366 0.9933263 ]
 [0.61255974 0.38744026]
 [0.00000049 0.9999995 ]
 [0.10626524 0.89373475]
 [0.17375524 0.8262448 ]
 [0.07897495 0.9210251 ]
 [0.9999491  0.00005092]
 [0.06008614 0.93991387]
 [1.         0.        ]
 [0.17811495 0.8218851 ]
 [0.37536386 0.6246361 ]
 [0.682633   0.317367  ]
 [0.07904339 0.9209566 ]] (16.198 sec)
INFO:tensorflow:loss = 0.27807373, step = 401 (32.447 sec)
INFO:tensorflow:probabilities = [[0.13600269 0.8639973

INFO:tensorflow:global_step/sec: 3.08952
INFO:tensorflow:probabilities = [[0.00195443 0.99804556]
 [0.         1.        ]
 [0.36455244 0.63544756]
 [0.00000354 0.9999964 ]
 [1.         0.        ]
 [0.00000106 0.9999989 ]
 [0.         1.        ]
 [0.82869315 0.17130685]
 [0.         1.        ]
 [0.99944955 0.0005504 ]
 [0.         1.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.9919809  0.00801914]
 [0.97346514 0.02653485]
 [0.9999963  0.00000367]
 [0.9997253  0.00027475]
 [0.         1.        ]
 [0.00004922 0.99995077]
 [0.9998958  0.00010419]
 [0.8473106  0.1526894 ]
 [0.         1.        ]
 [0.9992712  0.00072878]
 [1.         0.        ]
 [0.9952271  0.00477292]
 [0.06999119 0.9300088 ]
 [1.         0.        ]
 [0.08496927 0.9150307 ]
 [0.05462964 0.9453704 ]
 [0.9999691  0.00003092]
 [0.99999297 0.000007  ]
 [1.         0.00000003]] (16.156 sec)
INFO:tensorflow:loss = 0.03337487, step = 901 (32.366 sec)
INFO:tensorflow:probabilities = [[0.32198316 0.6780168

INFO:tensorflow:loss = 0.85180414, step = 1
INFO:tensorflow:probabilities = [[0.7206816  0.2793184 ]
 [0.32937673 0.6706233 ]
 [0.78001034 0.21998966]
 [0.6469098  0.3530902 ]
 [0.66762495 0.33237505]
 [0.37249067 0.6275093 ]
 [0.6238099  0.37619013]
 [0.7470516  0.2529484 ]
 [0.760321   0.23967901]
 [0.43481266 0.56518734]
 [0.85795635 0.14204362]
 [0.42858315 0.5714169 ]
 [0.7489607  0.25103936]
 [0.61234397 0.387656  ]
 [0.72970897 0.270291  ]
 [0.64632875 0.35367128]
 [0.67963535 0.32036465]
 [0.71927834 0.28072172]
 [0.5855416  0.41445845]
 [0.73061544 0.2693845 ]
 [0.38378686 0.61621314]
 [0.35474753 0.6452524 ]
 [0.6102556  0.38974434]
 [0.6152159  0.38478413]
 [0.35569373 0.64430624]
 [0.65935475 0.34064528]
 [0.4381973  0.5618026 ]
 [0.79046935 0.20953062]
 [0.56854    0.43146002]
 [0.6872189  0.3127811 ]
 [0.72640014 0.27359983]
 [0.42574465 0.57425535]] (16.110 sec)
INFO:tensorflow:global_step/sec: 3.1163
INFO:tensorflow:probabilities = [[0.61220074 0.3877993 ]
 [0.748844   

INFO:tensorflow:loss = 0.3078877, step = 501 (32.305 sec)
INFO:tensorflow:probabilities = [[0.22990899 0.77009106]
 [0.00396116 0.9960388 ]
 [0.29731455 0.7026854 ]
 [0.         1.        ]
 [0.00759544 0.9924045 ]
 [0.23379736 0.7662026 ]
 [0.22332168 0.7766783 ]
 [0.205132   0.79486805]
 [0.00077998 0.99922   ]
 [0.01338582 0.98661417]
 [0.7802055  0.21979448]
 [0.18979079 0.8102092 ]
 [1.         0.        ]
 [0.99999774 0.00000221]
 [0.00068003 0.99932003]
 [0.9959138  0.00408623]
 [0.21558349 0.7844165 ]
 [0.23846579 0.76153415]
 [0.000276   0.999724  ]
 [0.18725261 0.81274736]
 [0.9995555  0.00044441]
 [0.00737983 0.99262017]
 [0.20552322 0.7944768 ]
 [0.00000021 0.99999976]
 [0.36415908 0.6358409 ]
 [0.7542575  0.2457425 ]
 [0.9513279  0.04867204]
 [0.00050198 0.999498  ]
 [0.         1.        ]
 [0.00135805 0.9986419 ]
 [0.00024157 0.9997584 ]
 [0.16508156 0.8349185 ]] (16.116 sec)
INFO:tensorflow:global_step/sec: 3.09856
INFO:tensorflow:probabilities = [[0.9999999  0.00000015

INFO:tensorflow:loss = 0.0076046074, step = 1001 (32.210 sec)
INFO:tensorflow:probabilities = [[0.14125457 0.8587454 ]
 [0.00044296 0.9995571 ]
 [0.7286768  0.2713232 ]
 [0.00948023 0.99051976]
 [0.00539869 0.99460137]
 [0.9999925  0.00000751]
 [0.00377367 0.9962263 ]
 [0.00354925 0.9964508 ]
 [0.00018371 0.99981636]
 [1.         0.00000005]
 [0.00005247 0.99994755]
 [0.0003685  0.9996315 ]
 [0.00002926 0.9999708 ]
 [0.9987698  0.00123017]
 [1.         0.00000002]
 [1.         0.        ]
 [0.00000995 0.9999901 ]
 [0.9999994  0.00000063]
 [1.         0.00000002]
 [0.27790707 0.72209287]
 [0.1929177  0.80708236]
 [0.16742374 0.8325763 ]
 [0.         1.        ]
 [0.38672042 0.6132796 ]
 [0.00097419 0.9990258 ]
 [0.0189758  0.98102415]
 [0.00041181 0.9995882 ]
 [1.         0.        ]
 [0.9997397  0.00026033]
 [0.08528902 0.914711  ]
 [0.97901344 0.02098657]
 [1.         0.00000001]] (16.187 sec)
INFO:tensorflow:global_step/sec: 3.0754
INFO:tensorflow:probabilities = [[1.         0.     

In [12]:
print(total_res['4'])

0.96612024


In [ ]:
aa = []
a = np.array(([3,1,2,4]))
b = np.array(([5,65,12.1]))

In [ ]:
c = np.append(aa,b)
print(a)
print(c[0])

In [ ]:
"""
ev_results = np.zeros((len(ev_label), 1))


# Evaluate the model and print results
for i in range(len(ev_label)):
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": ev_data[i:i+1]},
        y=ev_label[i:i+1],
        num_epochs=1,
        shuffle=False)
    eval_results = net_classifier.evaluate(input_fn=eval_input_fn)

    ev_results[i] = eval_results["accuracy"]
    
"""

In [ ]:
#print(np.mean(ev_results))